In [6]:
!python video_detector.py --video venice.mp4 --det det

Loading network.....
Network successfully loaded
video_detector.py:115: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  output = model(Variable(img, volatile = True), CUDA)
/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
2.0070059299468994
FPS of the video is  0.50
4.074826002120972
FPS of the video is  0.49
5.805264949798584
FPS of the video is  0.52
7.496501922607422
FPS of the video is  0.53
9.2307288646698
FPS of the video is  0.54
11.033302068710327
FPS of the video is  0.54
12.932623863220215
FPS of the video is  0.54
14.8627450466156
FPS of the video is  0.54
16.644646883010864
FPS of the video is  0.54
18.421317100524902


In [2]:
model = Darknet("cfg/yolov3.cfg")
model.load_weights("yolov3.weights")

In [3]:
model = Darknet("cfg/yolov3.cfg")
inp = get_test_input()
print(inp.shape)
pred = model(inp, torch.cuda.is_available())
print (pred)

torch.Size([1, 3, 416, 416])


/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


tensor([[[ 18.4085,  16.3513, 127.6132,  ...,   0.6078,   0.5811,   0.5479],
         [ 14.0165,  16.0823, 130.0517,  ...,   0.4999,   0.4629,   0.5191],
         [ 13.0312,  18.1281, 363.9693,  ...,   0.5044,   0.6073,   0.4720],
         ...,
         [411.8825, 411.6734,   7.1775,  ...,   0.4133,   0.4810,   0.6057],
         [412.1896, 411.8992,  15.8745,  ...,   0.5578,   0.4719,   0.4259],
         [412.1518, 412.2277,  27.0367,  ...,   0.4285,   0.4651,   0.5174]]])


In [4]:
prediction = pred

In [6]:
p = write_results(prediction, confidence=0.5, num_classes =80, nms_conf = 0.4)

In [7]:
p.shape

torch.Size([1421, 8])

In [17]:
(prediction[:,:,4] > confidence).float().unsqueeze(2).shape

torch.Size([1, 10647, 1])

In [25]:
prediction.shape

torch.Size([1, 10647, 85])

In [24]:
prediction.size()

torch.Size([1, 10647, 85])

In [14]:
confidence = 0.5
thres_prediction = prediction * ((prediction[:,:,4] > confidence).float().unsqueeze(2))
def transform_box(b_x, b_y, b_w, b_h):
    """
    transform box from (b_x, b_y, b_w, b_h to c_top_x, c_top_y, c_bot_x, c_bot_y
    c_top_x, c_top_y : coordinates of top-left corner of box
    c_bot_x, c_bot_y : coordinates of bottom-right corner of box
    """
    c_top_x = b_x - b_w/2
    c_top_y = b_y - b_h/2
    
    c_bot_x = b_x + b_w/2
    c_bot_y = b_y + b_h/2
    
    
    return c_top_x, c_top_y, c_bot_x, c_bot_y
box_corner = thres_prediction.new(thres_prediction.shape)

box_corner[:,:,0], box_corner[:,:,1], box_corner[:,:,2], box_corner[:,:,3] = transform_box(thres_prediction[:,:,0],thres_prediction[:,:,1],thres_prediction[:,:,2], thres_prediction[:,:,3])

prediction[:,:,:4] = box_corner[:,:,:4]
    

In [10]:
dic = parse_cfg("cfg/yolov3.cfg")

In [11]:
set([x['type'] for x in dic])

{'convolutional', 'net', 'route', 'shortcut', 'upsample', 'yolo'}

In [44]:
yolo_layers = [x for x in dic if x['type']=='yolo']
print(len(yolo_layers))
yolo_key =[ x.keys() for x in yolo_layers]
print('##############################')
print([ x['mask'] for x in yolo_layers])
yolo_key

3
##############################
['6,7,8', '3,4,5', '0,1,2']


[dict_keys(['type', 'mask', 'anchors', 'classes', 'num', 'jitter', 'ignore_thresh', 'truth_thresh', 'random']),
 dict_keys(['type', 'mask', 'anchors', 'classes', 'num', 'jitter', 'ignore_thresh', 'truth_thresh', 'random']),
 dict_keys(['type', 'mask', 'anchors', 'classes', 'num', 'jitter', 'ignore_thresh', 'truth_thresh', 'random'])]

In [16]:
convolutional_layers = [x for x in dic if x['type']=='convolutional']
print(len(convolutional_layers))
convolutional_key =[ x.keys() for x in convolutional_layers]
print(convolutional_key)
print('##############################')
print('##############################')
batch_layers = [x for x in  convolutional_layers if 'batch_normalize' in (list(x.keys()))]
print([ x['batch_normalize'] for x in batch_layers])

75
[dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']), dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']), dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']), dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']), dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']), dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']), dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']), dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']), dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']), dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']), dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']), dict_k

In [30]:
shortcut_layers = [x for x in dic if x['type']=='shortcut']
print(len(shortcut_layers))
shortcut_key =[ x.keys() for x in shortcut_layers]
print(shortcut_key)
print('##############################')
print([ x['from'] for x in shortcut_layers])
print('##############################')
print([ x['activation'] for x in shortcut_layers])

23
[dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation']), dict_keys(['type', 'from', 'activation'])]
#######

In [ ]:
print([ x['activation'] for x in shortcut_layers])

In [21]:
upsample_layers = [x for x in dic if x['type']=='upsample']
print(len(upsample_layers))
upsample_key =[ x.keys() for x in upsample_layers]
upsample_key

2


[dict_keys(['type', 'stride']), dict_keys(['type', 'stride'])]

In [7]:
route_index = [i for i in range(len(dic)) if dic[i]['type']=='route']
route_index

[84, 87, 96, 99]

In [5]:
route_layers = [x for x in dic if x['type']=='route']
print(len(route_layers))
route_key =[ x.keys() for x in route_layers]
route_l = [x['layers'].split(',') for x in route_layers]
route_l

4


[['-4'], ['-1', ' 61'], ['-4'], ['-1', ' 36']]

In [15]:
conv_key =[ x.keys() for x in conv_layers]
conv_key

[dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']),
 dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']),
 dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']),
 dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']),
 dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']),
 dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']),
 dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']),
 dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']),
 dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']),
 dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']),
 dict_keys(['type', 'batch_normalize', 'filters', 'size', 'stride', 'pad', 'activation']),

In [5]:
dic

[{'type': 'net',
  'batch': '64',
  'subdivisions': '16',
  'width': '608',
  'height': '608',
  'channels': '3',
  'momentum': '0.9',
  'decay': '0.0005',
  'angle': '0',
  'saturation': '1.5',
  'exposure': '1.5',
  'hue': '.1',
  'learning_rate': '0.001',
  'burn_in': '1000',
  'max_batches': '500200',
  'policy': 'steps',
  'steps': '400000,450000',
  'scales': '.1,.1'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '32',
  'size': '3',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '64',
  'size': '3',
  'stride': '2',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '32',
  'size': '1',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '64',
  'size': '3',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'shortcut', 'from': '-3', 'activatio